In [21]:
import re
import json
from models.chat_model import ChatModel
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [23]:
bcolors = {
    'purple': '\033[95m',
    'blue': '\033[94m',
    'cyan' : '\033[96m',
    'green' :'\033[92m',
    'yellow' : '\033[93m',
    'red' : '\033[91m',
    'ENDC' : '\033[0m',
    'bold' : '\033[1m',
    'underline' : '\033[4m'
 }
    

def pprint(message, color=None):
    if color is None:
        print(message)
    else:
        pcolor = bcolors[color] if color in bcolors else bcolors['ENDC']
        print(f"{pcolor}{message}{bcolors['ENDC']}")
    

In [24]:
PREFIX = """Respond to the human as helpfully and accurately as possible. You have access to the following tools:"""
FORMAT_INSTRUCTIONS = """Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}}}
```"""
SUFFIX = """Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation:.
Thought:"""

HUMAN_MESSAGE_TEMPLATE = "{input}\n\n{agent_scratchpad}"

In [25]:
class PromptTemplate(object):
    def __init__(self, input_variables, messages):
        self.input_variables = input_variables
        self.messages = messages

    def format(self, **kwargs):
        messages = []
        for message in self.messages:
            text = message['template'].format(**kwargs)
            messages.append({
                "role": message['role'],
                "content": text
            })
        return messages

In [26]:
class Tool(object):
    def __init__(self, **kwargs):
        self.name = kwargs.get('name')
        self.func = kwargs.get('func')
        self.description = kwargs.get('description')
        self.args = kwargs.get('args')

    def run(self, args, verbose=False):
        if verbose:
            print(f"Running {self.name} with args {args}")
        return self.func(args)

In [27]:
def member_func(args):
    name = args.get('name')
    members = {
        "James": "James is a member of A team",
        "Jim": "Jim is a member of B team",
        "Jimmy": "Jimmy is a member of C team",
        "Jimothy": "Jimothy is a member D the team",
    }
    return f"{members[name]}"


member_tool = Tool(**{
    "name": "TeamQuery",
    "func": member_func,
    "description": "useful for when you want to know someone's team",
    "args": {'name': {'title': 'Name', 'type': 'string'}}
})

In [28]:
class Agent(object):
    def __init__(self, tools, stop=['Observation:'], max_iterations=15, **kwargs):
        self.model = ChatModel()
        self._stop = stop
        self.name_to_tool_map = self._re_tools(tools)
        self.prompt = self.create_prompt(tools)
        self.max_iterations = max_iterations
        self.observation_prefix = kwargs.get(
            'observation_prefix', "Observation: ")
        self.llm_prefix = kwargs.get('llm_prefix', "Thought:")
        self.verbose = kwargs.get('verbose', False)
        self.show_prompt = kwargs.get('show_prompt', False)

    def create_prompt(
        self,
        tools,
        prefix: str = PREFIX,
        suffix: str = SUFFIX,
        human_message_template: str = HUMAN_MESSAGE_TEMPLATE,
        format_instructions: str = FORMAT_INSTRUCTIONS,
        input_variables=None
    ):
        tool_strings = []
        for tool in tools:
            args_schema = re.sub(
                "}", "}}}}", re.sub("{", "{{{{", str(tool.args)))
            tool_strings.append(
                f"{tool.name}: {tool.description}, args: {args_schema}")
        formatted_tools = "\n".join(tool_strings)
        tool_names = ", ".join([tool.name for tool in tools])
        format_instructions = format_instructions.format(tool_names=tool_names)
        template = "\n\n".join(
            [prefix, formatted_tools, format_instructions, suffix])
        if input_variables is None:
            input_variables = ["input", "agent_scratchpad"]

        messages = [{
            "role": "system",
            "template": template,
        }, {
            "role": "user",
            "template": human_message_template
        }]

        prompt = PromptTemplate(input_variables, messages)
        return prompt

    def _re_tools(self, tools):
        name_to_tool_map = {}
        for tool in tools:
            name_to_tool_map[tool.name] = tool
        return name_to_tool_map

    def _construct_scratchpad(self, intermediate_steps):
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\n{self.observation_prefix}{observation}\n{self.llm_prefix}"
        if thoughts:
            return (
                f"This was your previous work "
                f"(but I haven't seen any of it! I only see what "
                f"you return as final answer):\n{thoughts}"
            )
        else:
            return thoughts

    def _output_parse(self, text):
        try:
            action_match = re.search(r"```(.*?)```?", text, re.DOTALL)
            if action_match is not None:
                response = json.loads(
                    action_match.group(1).strip().replace('json\n', '\n'), strict=False)
                if isinstance(response, list):
                    # gpt turbo frequently ignores the directive to emit a single action
                    response = response[0]
                return dotdict({
                    "type": "action",
                    "tool": response["action"],
                    "tool_input": response.get("action_input", {}),
                    "state" : 'finished' if response["action"] == 'Final Answer' else 'intermediate',
                    "output": response.get("action_input", {}),
                    "log": text
                })
            else:
                return dotdict({
                    "type": "action",
                    "state": "finished",
                    "output": text,
                    "log": text
                })
        except Exception as e:
            raise Exception(f"Could not parse LLM output: {text}") from e

    def get_full_inputs(self, intermediate_steps, **kwargs):
        """Create the full inputs for the llm from intermediate steps."""
        thoughts = self._construct_scratchpad(intermediate_steps)
        new_inputs = {"agent_scratchpad": thoughts, "stop": self._stop}
        full_inputs = {**kwargs, **new_inputs}
        return full_inputs

    def prep_prompts(self, input_list):
        """Prepare prompts from inputs."""
        stop = None
        if "stop" in input_list[0]:
            stop = input_list[0]["stop"]
        prompts = []
        for inputs in input_list:
            selected_inputs = {k: inputs[k]
                               for k in self.prompt.input_variables}
            prompt = self.prompt.format(**selected_inputs)
            prompts.append(prompt)

        return prompts, stop

    def _track_steps_verbose(self, step_output):
         if self.verbose:
                if 'input' in step_output:
                    pprint(f"> Start: \n{step_output['input']}\n", 'bold')   
                else:
                    action, observation = step_output
                    though = action.log
                    if not though.startswith(self.llm_prefix):
                        though = f"{self.llm_prefix}{though}"
                    pprint(f"{though}", 'green')
                    pprint(f"{self.observation_prefix}{observation}\n", 'blue')
                    if action.tool == 'Final Answer':
                        pprint(f"> Finished: \n{observation}\n", 'bold')

    def run(self, **kwargs):
        intermediate_steps = []
        iterations = 0
        self._track_steps_verbose(kwargs)
        while iterations <= self.max_iterations:
            next_step_outputs = self._take_next_step(intermediate_steps, kwargs)
            next_step_output = next_step_outputs[0]
            self._track_steps_verbose(next_step_output)
            if (next_step_output[0].state == 'finished'):
                return next_step_output[1]

            intermediate_steps.extend(next_step_outputs)
            iterations += 1

    def _take_next_step(self, intermediate_steps, inputs):
        full_inputs = self.get_full_inputs(intermediate_steps, **inputs)

        prompts, stop = self.prep_prompts([full_inputs])

        if self.show_prompt:
            messages = [message['content'] for message in prompts[0]]
            print("\n".join(messages))

        response = self.model(messages=prompts[0], stop=stop)
        output = self._output_parse(response)

        if (output.state == 'finished'):
            return [(output, output.output)]
        actions = []
        if (output.tool is not None):
            actions = [output]
        else:
            actions = output
        result = []
        for action in actions:
            if action.tool in self.name_to_tool_map:
                tool = self.name_to_tool_map[action.tool]
                observation = tool.run(action.tool_input)
                result.append((action, observation))
        return result

In [29]:
input = 'James belongs to which team?'
tools = [member_tool]
agent = Agent(tools, verbose=True)

In [30]:
agent.run(input=input)

> Start: 
James belongs to which team?

Thought: I can use the TeamQuery tool to find out which team James belongs to.
Action:
```
{
  "action": "TeamQuery",
  "action_input": {
    "name": "James"
  }
}
```

Observation: James is a member of A team
Thought:James belongs to team A.
Action:
```
{
  "action": "Final Answer",
  "action_input": "James belongs to team A."
}
```

Observation: James belongs to team A.
> Finished: 
James belongs to team A.

James belongs to team A.
